In [ ]:
import cartopy
import cartopy.crs as ccrs
import fiona
import geopandas
import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt
import pandas
import rasterio
import rasterio.plot

import glob
import json
import os

matplotlib.rcParams["figure.figsize"] = (8,8)
matplotlib.rcParams["figure.dpi"] = 72

In [ ]:
# read shared data folder location from config
with open("config.json", 'r') as fh:
    base_folder = json.load(fh)['data_folder']

In [ ]:
# read Arc LADs
lads_path = os.path.join(base_folder, 'GIS Data', 'arc_lad_uk16.gpkg')
lads_df = geopandas.read_file(lads_path)

In [ ]:
lad_centroids_df = lads_df[lads_df.in_arc == 1]
lad_centroids_df["geometry"] = lad_centroids_df.geometry.centroid
lad_centroids_df = lad_centroids_df[["desc", "geometry"]].rename(columns={"desc":"label"})
lad_centroids_df.to_file(
    os.path.join(base_folder, 'Visual Narrative', 'Data', 'arc_lad_centroids.geojson'), 
    driver="GeoJSON"
)
lad_centroids_df.head()

In [ ]:
# read Arc MSOAs
msoa_path = os.path.join(base_folder, 'GIS Data', 'msoa_arc.gpkg')
msoa_df = geopandas.read_file(msoa_path)

In [ ]:
msoa_names = pandas.read_csv(
    os.path.join(base_folder, 'Visual Narrative', 'Data', 'MSOA-Names-1.5.0.csv')
)[["msoa11cd", "msoa11hclnm"]]
msoa_names.head(1)

In [ ]:
msoa_centroids_df = msoa_df.copy()
msoa_centroids_df["geometry"] = msoa_centroids_df.geometry.centroid
msoa_centroids_df = msoa_centroids_df[["msoa11cd", "geometry"]].set_index("msoa11cd")
msoa_centroids_df = msoa_centroids_df \
    .join(msoa_names.set_index("msoa11cd"), how="left") \
    .rename(columns={'msoa11hclnm': 'label'})
msoa_centroids_df.to_file(
    os.path.join(base_folder, 'Visual Narrative', 'Data', 'arc_msoa_centroids.geojson'), 
    driver="GeoJSON"
)
msoa_centroids_df.head(1)

In [ ]:
# Natural Capital geodatabase location
nc_path = os.path.join(
    base_folder, 'Scenarios', 'Natural Capital', 'OxCamArc_Freedata_LADs', 'NatCap_Arc_FreeData_LADs.gdb')

In [ ]:
# Read each natural capital layer - takes >6G RAM
# dfs = []
# for layername in fiona.listlayers(nc_path):
#     if 'NatCap' in layername:
#         df = geopandas.read_file(nc_path, layer=layername)
#         dfs.append(df)
#         break
# nc_df = pandas.concat(dfs, axis=1)
# nc_df.columns

In [ ]:
density_path = os.path.join(
    base_folder, 'Scenarios', 'Density', 'Density_Surfaces', '*.asc')

In [ ]:
arc_df = geopandas.read_file(os.path.join(
    base_folder, 'Scenarios', 'Density', 'Density_Surfaces', 'arc_region', 'arc_region.shp'))

In [ ]:
arc_extent = (418_000, 573_000, 170_000, 325_000)
cty_extent = (475_000, 500_000, 225_000, 250_000)
ngb_extent = (487_000, 492_000, 231_500, 236_500)
extents = {
    'arc': arc_extent,
    'cty': cty_extent,
    'ngb': ngb_extent
}

In [ ]:
def plot_map(raster, raster_extent, extent, cmap='Greens', norm=None):
    osgb = ccrs.epsg(27700)
    ax = plt.axes([0., 0., 1., 1.], projection=osgb) # axes to fill image 
    ax.set_extent(extent, crs=osgb)
    ax.set_frame_on(False) # don't draw axes outline/background rectangle

    # add the image
    if raster is not None:
        ax.imshow(raster, origin='upper', extent=raster_extent, transform=osgb, cmap=cmap, norm=norm)
#     ax.add_geometries(
#         arc_df['geometry'], crs=osgb, 
#         edgecolor='black', linewidth=1, facecolor='#00000000')

    return ax

In [ ]:
# look at looping over first to get consistent vmax
#norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

for fname in glob.glob(density_path):
    if 'Nat' in fname or 'Dwellings' in fname:
        with rasterio.open(fname) as ds:
            data = ds.read(1)
            data_extent = rasterio.plot.plotting_extent(ds)
            
        if 'Dwellings' in fname:
            color = '#2D9CDB' # blue
        else:
            color = '#219653' # green

        cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white",color])
        
        if 'Nat' in fname:
            for zoom in ('arc', 'cty', 'ngb'):
                plot_map(data, data_extent, extents[zoom], cmap=cmap)
                plt.savefig(f"natcap_{zoom}.png")            
        else:
            policy, dwellings, _ = os.path.basename(fname).lower().split('_')
            if dwellings == 'expansion':
                dwellings = 'exp'
            elif dwellings == 'settlements':
                dwellings = 'set'
            else:
                assert False, dwellings

            for zoom in ('arc', 'cty', 'ngb'):
                plot_map(data, data_extent, extents[zoom], cmap=cmap)
                plt.savefig(f"density_{policy}_{dwellings}_{zoom}.png")
                        
        print(fname)

In [ ]:
udm_path = os.path.join(
    base_folder, 'Scenarios', 'UDM', 'ATI FINAL', '**', '*.asc')

In [ ]:
for fname in glob.glob(udm_path):
    with rasterio.open(fname) as ds:
        data = ds.read(1)
        data_extent = rasterio.plot.plotting_extent(ds)
        
    if 'suit' in fname:
        dwellings, policy, quantity = os.path.basename(fname).replace('.asc', '').lower().split('_')
        out_name = f"suitability_{policy}_{dwellings}_zoom.png"
    else:
        rate, dwellings, policy, quantity = os.path.basename(fname).replace('.asc', '').lower().split('_')
        if quantity == "dev":
            out_name = f"development_{policy}_{dwellings}_{rate}_zoom.png"
        if quantity == "dwell":     
            out_name = f"dwellings_{policy}_{dwellings}_{rate}_zoom.png"
    
        
    if quantity == 'suit':
        color = '#F2C94C' # yellow
    elif quantity == 'dev':
        color = '#EB5757' # red
    elif quantity == 'dwell':
        color = '#2D9CDB' # blue
    else:
        assert False, quantity
        
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white",color])

    
    for zoom in ('arc', 'cty', 'ngb'):
        plot_map(data, data_extent, extents[zoom], cmap=cmap)
        plt.savefig(out_name.replace('zoom', zoom))
        
    print(fname)